# Introduzione a Python
Si veda: [https://docs.python.org/3/tutorial](https://docs.python.org/3/tutorial/)

## Principi della programmazione in Python
- Variabili e tipizzazione dinamica
- Assenza di informazione e valori <i>None</i>

In [ ]:
a = 4
b = 'questa è "una" stringa'
c = '6'

## Tipi elementari
- Stringhe
- Tipi numerici
- Date

In [ ]:
x = None
x is not None

## Esempio 1: Indovina un numero

- *La macchina pensa un numero*
    - *intero e entro un minimo e massimo*
- *La macchina chiede all'utente un numero*
- *La macchina compara il numero con quello da indovinare*
- La macchina fornisce un feedback all'utente
- Questa cosa va fatta fino a indovinare

In [1]:
import numpy.random as rn

In [ ]:
def think_number(minn=0, maxn=20):
    x = rn.randint(low=minn, high=maxn)
    return x

## Istruzioni condizionali
- Flusso di esecuzione delle istruzioni
- ``if``, ``elif`` e ``else``

In [ ]:
n = think_number()
i = input('dimmi un numero: ')
try:
    trial = int(i)
except ValueError:
    i = input('HO DETTO UN NUMERO! ')
    try:
        trial = int(i)
    except ValueError:
        i = 5
if trial == n:
    print("BRAVO")
elif trial < n:
    print("Numero troppo basso")
else:
    print("Numero troppo alto")
print(n)


## Cicli
- istruzione `while`
- istruzione `for`

In [ ]:
from IPython.display import clear_output
import time

In [ ]:
x = 0
while x <= 4:
    x = x + 1
    print(x)

In [ ]:
for x in range(2, 10, 2):
    print(x)

In [ ]:
n = think_number()
m = 6
for iteration in range(m):
    # clear_output(wait=True)
    print('Partita numero ', iteration)
    i = input('dimmi un numero: ')
    try:
        trial = int(i)
    except ValueError:
        i = input('HO DETTO UN NUMERO! ')
        try:
            trial = int(i)
        except ValueError:
            i = 5
    if trial == n:
        print("BRAVO")
        break
    elif trial < n:
        print("Numero troppo basso")
    else:
        print("Numero troppo alto")
    # time.sleep(10)
print(n)

## Simulatore
- modificare `number_game` per prendere input e dare output a un'altra funzione
- implementare una funzione per giocare
    - più stategie di gioco
    - raccolta statistiche (numero di tentativi per partita)

In [2]:
import numpy as np

In [ ]:
def init_game(start = 1, end = 10):
    n = rn.randint(start, end)
    return n

def evaluate_guess(trial, correct_number):
    return np.sign(trial - correct_number)

def random_player(start, end, correct_number, max_iterations):
    final = max_iterations
    for game in range(max_iterations):
        trial = rn.randint(start, end)
        feedback = evaluate_guess(trial, correct_number)
        if feedback == 0:
            final = game
            break
    return final

def one_player(start, end, correct_number, max_iterations):
    final = max_iterations
    for game in range(max_iterations):
        trial = rn.randint(start, end)
        feedback = evaluate_guess(trial, correct_number)
        if feedback == 0:
            final = game
            break
        if feedback < 0:
            start = trial
        else:
            end = trial
    return final

In [ ]:
s, e, i = 0, 100, 500
n = init_game(s, e)
history_rp = []
history_op = []
for test in range(1000):
    o = random_player(s, e, n, i)
    j = one_player(s, e, n, i)
    history_rp.append(o)
    history_op.append(j)
O = np.array([history_rp, history_op]).T

In [ ]:
s, e, i = 0, 100, 500
n = init_game(s, e)
O = np.array([[random_player(s, e, n, i), one_player(s, e, n, i)] for x in range(1000)])

In [ ]:
O

In [ ]:
O.mean(axis=0)

In [ ]:
sum(history_rp) / len(history_rp)

In [ ]:
sum(history_op) / len(history_op)

In [3]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), ncols=2)
ax[0].plot(np.cumsum(O[:,0]), label='Random player')
ax[0].plot(np.cumsum(O[:,1]), label='One player')
ax[1].scatter(O[:,0], O[:,1], alpha=0.4)
ax[0].legend()
plt.show()

## Liste

In [ ]:
s = 0
a = [[2, 3, 6], [6, 7, 9]]
print(a[1][1])

In [ ]:
e = [x**2 for x in range(0, 11, 2) if x != 4]
e

In [ ]:
b = np.array(a)

In [ ]:
print(b)

In [ ]:
b.shape

In [ ]:
b.dot(b.T)

In [ ]:
b.mean(axis=1)

In [ ]:
print(b[:,-2:])

In [ ]:
st = 'obgect does not support'
g = list(st)
g[2] = 'j'
st = "".join(g)

In [ ]:
st

In [ ]:
";".join(['e', 'p'])

# Programmazione a oggetti

`super(SubClass,self).__init__( x )`

**Banco**
- metodi
    - genera un numero
    - verifica i tentativi
- proprietà
    - numero da indovinare
    - range

**Giocatore**
- metodi
    - tentare (aggiustare il tiro)
    - ricevere un feedback
- proprietà
    - range della partita
    - numero dei tentativi (history)

In [45]:
class Dealer(object):
    
    def __init__(self, start=0, end=10, players=None):
        self.s = start
        self.e = end
        self.n = None
        self.deal()
        if players is None:
            self.players = []
        else:
            self.players = players
        
    def __str__(self):
        return "Dealer with start {} and end {}".format(self.s, self.e)
    
    def deal(self):
        self.n = rn.randint(self.s, self.e)
        
    def check(self, trial):
        return np.sign(trial - self.n)
    
    def game(self, max_iterations):
        for p in self.players:
            p.s, p.e = self.s, self.e
        for i in range(max_iterations):
            for p in self.players:
                if not p.win:
                    trial = p.guess()
                    feedback = self.check(trial)
                    p.get_feedback(feedback, num_trials=i, trial=trial)
        for p in self.players:
            if not p.win:
                p.history.append(max_iterations)
    
    def simulation(self, max_iterations, num_sim=1000):
        for s in range(num_sim):
            for p in self.players:
                p.win = False
                p.current_feedback = None
            self.game(max_iterations)
    

class Player(object):
    
    def __init__(self):
        self.s, self.e = None, None
        self.current_feedback = None
        self.win = False
        self.history = []
    
    def guess(self):
        return rn.randint(self.s, self.e)
    
    def get_feedback(self, feedback, num_trials, trial):
        self.current_feedback = feedback
        if self.current_feedback == 0:
            self.win = True
            self.history.append(num_trials)

            
class SmartPlayer(Player):
    
    def __init__(self):
        super(SmartPlayer, self).__init__()
    
    def get_feedback(self, feedback, num_trials, trial):
        self.current_feedback = feedback
        if self.current_feedback == 0:
            self.win = True
            self.history.append(num_trials)
        elif self.current_feedback < 0:
            self.s = trial
        else:
            self.e= trial


class OnePlayer(Player):
    
    def __init__(self):
        super(onePlayer, self).__init__()
    

In [46]:
start, end = 0, 100
p = Player()
s = SmartPlayer()
x = Player()
y = SmartPlayer()
print(p.current_feedback, p.win, s.current_feedback, s.win)
d = Dealer(start, end, players=[p, s, x, y])
d.simulation(max_iterations=100, num_sim=1000)
print(p.current_feedback, p.win, s.current_feedback, s.win)

None False None False
-1 False 0 True


In [47]:
for k in [p, s, x, y]:
    print(np.array(k.history).mean())

62.067
7.783
62.614
7.772


In [44]:
isinstance(s, SmartPlayer)

True

## Filesystem e salvataggio file
- creare una cartella superiore che si chiama `stats_data`
- salvare in `stats_data` il file con i risultati che si chiamerà `guess_number`

In [ ]:
import os
import json

## Dictionary

## Gestione dell'errore
- Concetto di eccezione
- ``try``, ``except``, ``raise``

# Uso avanzato di liste e dizionari
- _List slice_ e _list comprehension_
- Il modulo <i>collections</i>: defaultdict e counter
- Iteratori, <i>enumerate</i>, cicli

## Input e output da file